# Setup 

In [ ]:
!rm -r /kaggle/working/*

In [ ]:
!cat /etc/os-release
!python -V

import os
import torch

print(f"GPUs ({torch.cuda.device_count()}):")
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_name(i))
os.environ['CPU_COUNT'] = str(os.cpu_count())

!echo $CPU_COUNT

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
password = UserSecretsClient().get_secret("gh_token")
os.environ['GITHUB_AUTH'] = 'marcodiri:' + password
!git clone https://$GITHUB_AUTH@github.com/marcodiri/s2s-contrastive-text-recognition.git ~/s2s

In [ ]:
!mkdir -p ~/s2s/datasets/iam/words
!tar zxf /kaggle/input/iamdatasethtrwords/words.tgz -C ~/s2s/datasets/iam/words

In [ ]:
%cd ~/s2s/datasets
!python process_iam.py

In [ ]:
!pip install -q fire
%cd ~/s2s/datasets
!python create_lmdb_dataset.py \
  --inputPath iam_train/ --gtFile iam_train/gt.txt --outputPath lmdb_train/IAM
!python create_lmdb_dataset.py \
  --inputPath iam_val/ --gtFile iam_val/gt.txt --outputPath lmdb_val/IAM
!python create_lmdb_dataset.py \
  --inputPath iam_test/ --gtFile iam_test/gt.txt --outputPath lmdb_test/IAM

# Training

In [ ]:
%cd ~/s2s
!git pull --all
!git checkout no-instance-mapping

In [ ]:
%cd ~/s2s
!pip install -q matplotlib lmdb torch torchvision pandas kornia==0.6.8 torchmetrics pytorch-lightning natsort importlib-metadata

### Pre-training

In [ ]:
%cd ~/s2s
!python pretrain.py \
--train_data ~/s2s/datasets/lmdb_train --select_data '/' \
--sensitive \
--num_iter 200000 --batch_size 192 --log_every_n_steps 10 \
--lr 10 --plateau 0.1 --patience 5 --earlystopping \
--workers 2 --save_dir '/kaggle/working/saved_models/' \
--temperature 0.07 \
--Transformation None --FeatureExtraction VGG --SequenceModeling BiLSTM --InstanceMapping None